# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [3]:
import nltk
from nltk.tokenize import word_tokenize

word_tokenize("dking sport")

['dking', 'sport']

In [4]:
import pandas as pd

# Load the bigrams dataset
file = 'bigrams.txt'
ngrams = pd.read_csv(file, sep='\t', header=None, index_col=None, encoding='latin')
ngrams[1] = ngrams[1].astype(str)
ngrams[2] = ngrams[2].astype(str)
ngrams

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and
...,...,...,...
1020380,24,zviad,gamsakhurdia
1020381,25,zweimal,leben
1020382,24,zwick,and
1020383,24,zydeco,music


In [5]:
from collections import Counter

# Build a vocabulary
vocab = Counter()
for i, row in ngrams.iterrows():
    for word in row[1:]:
        vocab[word] += row[0]
len(vocab)

68784

In [6]:
# Base of Norvig's solution

def candidates(word: str, early_known: bool = True) -> set[str]:
    "Generate possible spelling corrections for word."
    c = known([word])
    if not c or not early_known:
        c |= known(edits1(word))
    return (c or known(edits2(word)) or [word])

def known(words: list[str]): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in vocab) # Take our `vocab`

def edits1(word: str) -> set[str]:
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word: str) -> set[str]: 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [7]:
candidates('dking')

{'ding', 'doing', 'duking', 'dying', 'eking', 'king'}

In [8]:
def editsk(word, k):
    e = edits1(word)
    prev = e
    for i in range(1,k):
        temp = set(e2 for e1 in prev for e2 in edits1(e1))
        e.update(temp)
        prev = temp
    e.discard(word)
    return e

In [9]:
known(editsk('dking', 1))

{'ding', 'doing', 'duking', 'dying', 'eking', 'king'}

In [10]:
def ngram_weight(ngram: list[str]) -> float:
    "Prior number of times of the ngram occuring."
    # Assuming ngrams are sorted, apply binary search
    lo = 0
    hi = len(ngrams)
    while hi > lo:
        i = (lo + hi) // 2
        row = ngrams.iloc[i]
        cmp = 0
        for j in range(len(ngram)):
            if row[j + 1] < ngram[j]:
                cmp = -1
                break
            if row[j + 1] > ngram[j]:
                cmp = 1
                break
        if cmp == 0:
            return row[0]
        if cmp == -1:
            lo = i + 1
        else:
            hi = i
    return 0

In [11]:
ngram_weight(["dying", "patient"])

32

In [12]:
def correct_spelling(context: list[str], incorrect: int) -> list[tuple[int, str]]:
    "Returns correction suggestions (with attached weights) for the word in the context."
    suggestions = list(candidates(context[incorrect]))
    if len(suggestions) < 1:
        # No suggestions found
        return []

    # Consider corrections with their priors
    options = []
    max_weight = 0
    new_context = context.copy()
    for correction in suggestions:
        new_context[incorrect] = correction
        weight = ngram_weight(new_context)
        max_weight = max(max_weight, weight)
        options.append((weight, correction))

    if max_weight == 0:
        # No bigrams match
        if context[incorrect] in vocab:
            # The word is known, consider it not a mistake
            options = [(vocab[context[incorrect]], context[incorrect])]
        else:
            # Select by word prior
            for i in range(len(options)):
                options[i] = (vocab[options[i][1]], options[i][1])

    options.sort(reverse=True, key=lambda x: x[0])
    return options

In [13]:
correct_spelling(["dking", "patient"], 0)

[(32, 'dying'),
 (0, 'eking'),
 (0, 'duking'),
 (0, 'ding'),
 (0, 'king'),
 (0, 'doing')]

In [14]:
correct_spelling(["dking", "sports"], 0)

[(225735, 'doing'),
 (43773, 'king'),
 (19302, 'dying'),
 (180, 'ding'),
 (80, 'eking'),
 (63, 'duking')]

In [15]:
def autocorrect(sentence: str) -> str:
    "Attempts to autocorrect every word in the sentence"
    # Tokenize
    tokenized = word_tokenize(sentence)
    if len(tokenized) < 2:
        # TODO
        return sentence

    # Look for unknown words - likely mistakes
    corrected = tokenized.copy()
    for i in range(len(tokenized)):
        # if tokenized[i] not in vocab or (i + 1 < len(tokenized) and ngram_weight([tokenized[i], tokenized[i + 1]]) < 50):
            # Try to correct the word by looking at both bigrams
            correction = tokenized[i]
            correction_weight = 0
            if i > 0:
                weight, word = correct_spelling([tokenized[i - 1], tokenized[i]], 1)[0]
                if weight > correction_weight:
                    correction_weight = weight
                    correction = word
            if i + 1 < len(tokenized):
                weight, word = correct_spelling([tokenized[i], tokenized[i + 1]], 0)[0]
                if weight > correction_weight:
                    correction_weight = weight
                    correction = word
            corrected[i] = correction
    return ' '.join(corrected)

In [16]:
autocorrect("i liike spots")

'i like spots'

In [17]:
autocorrect("doing mth and solving problms")

'doing math and solving problems'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [19]:
# The test dataset link: https://www.kaggle.com/datasets/ved1104/wiki-sentences?resource=download
test_df = pd.read_csv('wiki_sentences_v2.csv')
test_df

,sentence
0,"confused and frustrated, connie decides to lea..."
1,"later, a woman’s scream is heard in the distance."
2,christian is then paralyzed by an elder.
3,the temple is set on fire.
4,"outside, the cult wails with him."
...,...
4313,"confidencial also responded negatively, callin..."
4314,and le parisien gave the film their highest fi...
4315,"the museum collection includes 37,000 film tit..."
4316,its predecessor was the dutch historical film ...


In [20]:
import string

def preprocess(s: str) -> str:
    # Remove punctuation and make lowercase
    translator = str.maketrans('', '', string.punctuation)
    return s.translate(translator).lower().strip()

In [21]:
# Preprocess the test dataset
test_df = test_df.apply(lambda row: preprocess(row[0]), axis=1).dropna()
test_df

/tmp/ipykernel_36483/3116923023.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_df = test_df.apply(lambda row: preprocess(row[0]), axis=1).dropna()


0       confused and frustrated connie decides to leav...
1         later a woman’s scream is heard in the distance
2                 christian is then paralyzed by an elder
3                               the temple is set on fire
4                         outside the cult wails with him
                              ...                        
4313    confidencial also responded negatively calling...
4314    and le parisien gave the film their highest fi...
4315    the museum collection includes 37000 film titl...
4316    its predecessor was the dutch historical film ...
4317        1920sfilmstar greta garbo by alexander binder
Length: 4318, dtype: object

In [22]:
import random

def generate_word_error(word: str, edit_distance: int = 1) -> str:
    "Replaces the word with a random one that is `edit_distance` away"
    options = list(editsk(word, edit_distance))
    return options[random.randint(0, len(options) - 1)]

def generate_errors(error_rate: float, edit_distance: int = 1):
    "Replaces the fixed ratio of the words with random errors `edit_distance` away"
    for sent in test_df:
        words = sent.split()
        errors = int(error_rate * len(words))
        error_indices = random.sample(range(len(words)), min(errors, len(words)))
        for i in error_indices:
            words[i] = generate_word_error(words[i], edit_distance)
        yield (sent, ' '.join(words))

In [23]:
class ConfusionMatrix:
    def __init__(self):
        self.true_pos = 0
        self.true_neg = 0
        self.false_pos = 0
        self.false_neg = 0

    def add(self, true: bool, positive: bool):
        if true:
            if positive:
                self.true_pos += 1
            else:
                self.true_neg += 1
        else:
            if positive:
                self.false_pos += 1
            else:
                self.false_neg += 1

    def true(self) -> int:
        return self.true_pos + self.true_neg

    def false(self) -> int:
        return self.false_pos + self.false_neg

    def total(self) -> int:
        return self.true() + self.false()
            
    def accuracy(self) -> float:
        return self.true() / self.total()

    def precision(self) -> float:
        return self.true_pos / (self.true_pos + self.false_pos)

    def recall(self) -> float:
        return self.true_pos / (self.true_pos + self.false_neg)

    def f1_score(self) -> float:
        prec = self.precision()
        recall = self.recall()        
        return 2 * (prec * recall) / (prec + recall)

    def __repr__(self) -> str:
        return f"""
         |  True  |  False  |
---------+--------+---------+
Positive |{self.true_pos:^8}|{self.false_pos:^9}|
Negative |{self.true_neg:^8}|{self.false_neg:^9}|
---------+--------+---------+

Accuracy : {self.accuracy():.2}
Precision: {self.precision():.2}
Recall   : {self.recall():.2}
F1 Score : {self.f1_score():.2}
        """

In [24]:
from tqdm import tqdm

# Test my implementation

confusion = ConfusionMatrix()

random.seed(69)
for true, error in tqdm(list(generate_errors(0.2))[:1000]): #tqdm(generate_errors(2), total=len(test_df)):
    expected = true.split()
    errored = error.split()
    corrected = autocorrect(error).split()
    for i in range(len(expected)):
        confusion.add(expected[i] == corrected[i], errored[i] != corrected[i])
confusion

100%|██████████| 1000/1000 [01:55<00:00,  8.64it/s]



         |  True  |  False  |
---------+--------+---------+
Positive |  1193  |   711   |
Negative |  8113  |   280   |
---------+--------+---------+

Accuracy : 0.9
Precision: 0.63
Recall   : 0.81
F1 Score : 0.71
        

In [25]:
# Norvig's solution

def P(word, N=sum(vocab.values())):
    "Probability of `word`."
    return vocab[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [26]:
correction('speling')

'spelling'

In [27]:
correction('korrectud')

'corrected'

In [28]:
def norvig_correct(sentence: str) -> str:
    corrected = [correction(word) for word in sentence.split()]
    return ' '.join(corrected)

In [29]:
norvig_correct("doing mth and solving problms")

'doing math and solving problems'

In [30]:
# Test Norvig's implementation

confusion = ConfusionMatrix()

random.seed(69)
for true, error in tqdm(list(generate_errors(0.2))[:1000]): #tqdm(generate_errors(2), total=len(test_df)):
    expected = true.split()
    errored = error.split()
    corrected = norvig_correct(error).split()
    for i in range(len(expected)):
        confusion.add(expected[i] == corrected[i], errored[i] != corrected[i])
confusion

100%|██████████| 1000/1000 [00:49<00:00, 20.38it/s]



         |  True  |  False  |
---------+--------+---------+
Positive |  1181  |   689   |
Negative |  8146  |   281   |
---------+--------+---------+

Accuracy : 0.91
Precision: 0.63
Recall   : 0.81
F1 Score : 0.71
        

In [31]:
def showcase(sent: str):
    print(f"Original   : {sent}")
    print(f"My solution: {autocorrect(sent)}")
    print(f"Norvig's   : {norvig_correct(sent)}")

# Implementation

My solution is based on the Norvig's solution and has a few changes to incorporate context. Each word is given in the context of both bigrams (to the left and to the right). In each bigram, Norvig's solution is applied with a small difference:
- Instead of the probability of the word occuring in the dataset, the probability of the bigram is used
- If none of the correction suggestions give bigrams with above 0 probability, unigram probability is used (i.e. Norvig's solution)

# Testing

For testing, I used the [Wiki Sentences](https://www.kaggle.com/datasets/ved1104/wiki-sentences?resource=download) dataset. It is preprocessed to get rid of punctuation and upper-case. Then, in each sample 20% of the words get randomly replaced by typos that are 1 edit distance away. The model is said to produce a true positive result if it correctly identifies and changes a typo to the original word.

# Results

| **My solution:** |  True  |  False  | **Norvig's solution** |  True  |  False  |
|---------|--------|---------|       ---------|--------|---------|
|Positive |  1193  |   711   |       Positive |  1181  |   689   |
|Negative |  8113  |   280   |       Negative |  8146  |   281   |
|Accuracy |  0.9   |         |       Accuracy |  0.91
|Precision|  0.63  |         |       Precision|  0.63
|Recall   |  0.81  |         |       Recall   |  0.81
|F1 Score |  0.71  |         |       F1 Score |  0.71

The results are very similar for both solution. The likely reason for that is that most errors in the generated test dataset are do not require context to correct. If we cherry-pick sample that do require context, however, we can see that Norvig's solution fails to correct mistakes properly.

In [33]:
showcase("dking patient")

Original   : dking patient
My solution: dying patient
Norvig's   : doing patient


In [34]:
showcase("dking sports")

Original   : dking sports
My solution: doing sports
Norvig's   : doing sports
